In [1]:
import pandas as pd
import numpy as np
import networkx as nx

In [ ]:
raw_reference = pd.read_csv('./scats_reference.csv', dtype={'Site_Number': np.int32, 'Location_Descriptors': str, 'Site_Type': str})
raw_reference = raw_reference[raw_reference.Site_Type == "INT"]
raw_reference = raw_reference.rename(columns={"Site_Number": "SCATS_Number"})
raw_reference = raw_reference.drop_duplicates()
raw_reference

,SCATS_Number,Location_Description,Site_Type
0,964,ABBOTTS/CLELANDS DEVELOPMENTS,INT
1,968,ABBOTTS/GAINE/MONASH,INT
2,972,ABBOTTS/NATIONAL,INT
3,983,ABBOTTS/REMINGTON,INT
4,1053,ABBOTTSFORD/HAINES,INT
...,...,...,...
4506,6075,WYNDHAM ST/VAUGHAN,INT
4507,4107,YAN YEAN/IRONBARK,INT
4509,5048,YARRA/EASTERN BEACH,INT
4510,5081,YARRA/LT MALOP,INT


In [3]:
raw_data = pd.read_csv('./scats_data.csv')
raw_data = raw_data.drop_duplicates()
raw_data

,SCATS_Number,Location,NB_LATITUDE,NB_LONGITUDE
0,970,WARRIGAL_RD N of HIGH STREET_RD,-37.867030,145.091590
31,970,HIGH STREET_RD E of WARRIGAL_RD,-37.867350,145.091950
62,970,WARRIGAL_RD S of HIGH STREET_RD,-37.867600,145.091460
93,970,HIGH STREET_RD W of WARRIGAL_RD,-37.867230,145.091030
123,2000,WARRIGAL_RD N of TOORAK_RD,-37.851683,145.094346
...,...,...,...,...
4042,4812,SWAN_ST SW of MADDEN_GV,-37.829030,145.014560
4068,4821,WALMER_ST N OF VICTORIA_ST,-37.812850,145.008490
4099,4821,VICTORIA_ST E OF BURNLEY_ST,-37.812930,145.008650
4130,4821,BURNLEY_ST S OF VICTORIA_ST,-37.813120,145.008440


In [4]:
ref_df = raw_reference.drop(columns={"Site_Type"})
data_df = raw_data.drop(columns={"Location"})
# Perform an inner merge to keep only SCATS numbers present in both tables
merged_df = pd.merge(ref_df, data_df, on='SCATS_Number', how='inner')
merged_df

,SCATS_Number,Location_Description,NB_LATITUDE,NB_LONGITUDE
0,4057,BALWYN/BELMORE,-37.80431,145.08197
1,4057,BALWYN/BELMORE,-37.80508,145.08246
2,4057,BALWYN/BELMORE,-37.80564,145.08171
3,4057,BALWYN/BELMORE,-37.80487,145.08092
4,3001,BARKERS/CHURCH/HIGH,-37.81441,145.02243
...,...,...,...,...
135,3682,WARRIGAL/RIVERSDALE,-37.83741,145.09626
136,4063,WHITEHORSE/BALWYN,-37.81404,145.08010
137,4063,WHITEHORSE/BALWYN,-37.81440,145.08045
138,4063,WHITEHORSE/BALWYN,-37.81476,145.07996


In [ ]:
def create_network_graph(_df: pd.DataFrame):

	# Initialize an undirected graph
	nodegraph = nx.Graph()

	# Add nodes (SCATS sites) with location description, latitude, and longitude
	for _, row in _df.iterrows():
		scats_num = row['SCATS_Number']
		location = row['Location_Description']
		latitude = row['NB_LATITUDE']
		longitude = row['NB_LONGITUDE']
		nodegraph.add_node(scats_num, location=location, latitude=latitude, longitude=longitude)

	# Parse streets and create edges between intersections sharing a street
	street_to_nodes = {}

	for _, row in _df.iterrows():
		scats_num = row['SCATS_Number']
		# Split the location description by '/' to get individual streets
		streets = row['Location_Description'].split('/')
		# Clean and process each street name
		streets = [street.strip() for street in streets]

		# Associate each street with the SCATS number
		for street in streets:
			if street:
				if street not in street_to_nodes:
					street_to_nodes[street] = []
				street_to_nodes[street].append(scats_num)

	# Create edges between SCATS sites that share a street
	for street, nodes in street_to_nodes.items():
		# If multiple intersections share a street, connect them
		for i in range(len(nodes)):
			for j in range(i + 1, len(nodes)):
				nodegraph.add_edge(nodes[i], nodes[j], street=street)

	# There is a non-zero chance that this ignores distance

	return nodegraph

networkgraph = create_network_graph(merged_df)

# Print basic graph info
print(f"Number of nodes (intersections): {networkgraph.number_of_nodes()}")
print(f"Number of edges (street connections): {networkgraph.number_of_edges()}")

# List all nodes and their attributes
for node, attrs in networkgraph.nodes(data=True):
	print(f"Node {node}: Location = {attrs['location']}, Lat = {attrs['latitude']}, Lon = {attrs['longitude']}")

# List all edges and their street
for u, v, attrs in networkgraph.edges(data=True):
	print(f"Edge {u} -- {v}: Street = {attrs['street']}")

# Save the graph to a file
#nx.write_graphml(G, "scats_graph.graphml")

Number of nodes (intersections): 40
Number of edges (street connections): 133
Node 4057: Location = BALWYN/BELMORE, Lat = -37.80487, Lon = 145.08092
Node 3001: Location = BARKERS/CHURCH/HIGH, Lat = -37.81457, Lon = 145.02161
Node 3002: Location = BARKERS/DENMARK/POWER, Lat = -37.8151, Lon = 145.02608
Node 4035: Location = BURKE/BARKERS/MONT ALBERT, Lat = -37.8188, Lon = 145.05739
Node 4032: Location = BURKE/BELMORE/HARP, Lat = -37.80225, Lon = 145.06081
Node 4040: Location = BURKE/CAMBERWELL/RIVERSDALE, Lat = -37.83254, Lon = 145.05507
Node 3120: Location = BURKE/CANTERBURY, Lat = -37.82284, Lon = 145.05684
Node 4034: Location = BURKE/COTHAM/WHITEHORSE, Lat = -37.81184, Lon = 145.05901
Node 4030: Location = BURKE/DONCASTER/KILBY/HIGH, Lat = -37.79471, Lon = 145.0612
Node 4043: Location = BURKE/TOORAK, Lat = -37.84714, Lon = 145.05205
Node 2000: Location = BURWOOD HWY/WARRIGAL, Lat = -37.85187, Lon = 145.09407
Node 4266: Location = BURWOOD/AUBURN, Lat = -37.82518, Lon = 145.04301
Node 4

In [6]:
import search

method = search.select_method("AS")
